In [34]:
import numpy as np
import pandas as pd
import re
import DataCleaning_Function as DC

# Cleaning Data

## Importing our dataset

In [2]:
data = pd.read_csv("us_perm_visas.csv",low_memory=False)

The following is a sample of our data. 

In [3]:
data[200100:200104]

,add_these_pw_job_title_9089,agent_city,agent_firm_name,agent_state,application_type,case_no,case_number,case_received_date,case_status,class_of_admission,...,ri_pvt_employment_firm_to,ri_us_workers_considered,schd_a_sheepherder,us_economic_sector,wage_offer_from_9089,wage_offer_to_9089,wage_offer_unit_of_pay_9089,wage_offered_from_9089,wage_offered_to_9089,wage_offered_unit_of_pay_9089
200100,NaN,New York,Law Offices of Harvey Shapiro,NEW YORK,NaN,NaN,A-15007-40058,2015-01-15,Certified,H-1B,...,NaN,NaN,N,NaN,"135,000.00",NaN,Year,NaN,NaN,NaN
200101,NaN,San Francisco,Werner & Associates,CALIFORNIA,NaN,NaN,A-14350-35169,2014-12-18,Certified,H-1B,...,NaN,NaN,N,NaN,"150,000.00","150,000.00",Year,NaN,NaN,NaN
200102,NaN,Minneapolis,"Fredrikson & Byron, P.A.",MINNESOTA,NaN,NaN,A-15039-49064,2015-03-31,Certified,H-1B,...,NaN,NaN,N,NaN,"71,531.00","89,000.00",Year,NaN,NaN,NaN
200103,NaN,NaN,NaN,NaN,NaN,NaN,A-14351-35492,2014-12-17,Certified,H-1B,...,NaN,NaN,N,NaN,"97,989.00",NaN,Year,NaN,NaN,NaN


## Removing Columns

We see that there are 374362 data points with 154 variables. We will remove the data points and variables that will not provide information relevant to our questions of interest. 

I. Using `pd.isnull().sum()`, can can see the variables where the majority of the rows are empty. We will carefully remove these columns. Since there will be a large amount of variables we will remove, we will first put the names of the variables that we are moving in a list and then drop them later from the dataframe. Note that we are not removing every single variable that have majority of empty rows. For example, `naics_2007_us_title` have 354462 empty rows, however, this column represents the same information in `naics_us_title`, `naics_us_title_2007` and `naics_title`. All three of the data points combined will cover almost every row in our dataset. Therefore, we need to be careful not to remove variables like these, but rather merge our dataset so they are just one column. 

In [4]:
b = data.isnull().sum()

In [5]:
# Sample of variables we deleted because majority of the rows are empty. 
b.sort_values(ascending=False).head(5)

orig_file_date                 374081
orig_case_no                   374068
recr_info_job_fair_to          372569
recr_info_job_fair_from        372564
recr_info_on_campus_recr_to    372497
dtype: int64

In [6]:
# Add the columns to delete in a list. 
drop_list = ['add_these_pw_job_title_9089','foreign_worker_info_alt_edu_experience',\
        'foreign_worker_info_birth_country','foreign_worker_info_postal_code',\
        'foreign_worker_info_rel_occup_exp','foreign_worker_info_req_experience',\
        'foreign_worker_info_training_comp','foreign_worker_ownership_interest',\
        'fw_info_alt_edu_experience','fw_info_birth_country',\
        'fw_info_postal_code','fw_info_rel_occup_exp',\
        'fw_info_req_experience','fw_info_training_comp',\
        'fw_ownership_interest','ji_foreign_worker_live_on_premises',\
        'ji_fw_live_on_premises','ji_offered_to_sec_j_foreign_worker',\
        'ji_offered_to_sec_j_fw','job_info_alt_cmb_ed_oth_yrs',\
        'job_info_alt_combo_ed','job_info_alt_combo_ed_other', \
        'job_info_alt_field_name','job_info_alt_occ',\
        'job_info_alt_occ_job_title','job_info_alt_occ_num_months',\
        'job_info_training_field','job_info_training_num_months',
        'orig_file_date','orig_case_no','recr_info_job_fair_to',\
        'recr_info_job_fair_from','recr_info_on_campus_recr_to',\
        'recr_info_on_campus_recr_from','ri_coll_teach_select_date',\
        'ri_coll_tch_basic_process','recr_info_coll_teach_comp_proc',\
        'recr_info_pro_org_advert_to','recr_info_pro_org_advert_from' ,\
        'recr_info_prof_org_advert_to','recr_info_prof_org_advert_from',\
        'pw_source_name_other_9089','ri_pvt_employment_firm_to',\
        'ri_pvt_employment_firm_from','ri_us_workers_considered',\
        'recr_info_radio_tv_ad_from','recr_info_radio_tv_ad_to',\
        'ri_campus_placement_to','ri_campus_placement_from',\
        'ri_employee_referral_prog_from','ri_employee_referral_prog_to',\
        'pw_job_title_908','recr_info_barg_rep_notified',\
        'ri_coll_teach_pro_jnl','ri_job_search_website_to',\
        'ri_job_search_website_from','preparer_info_title',\
        'pw_job_title_9089', 'recr_info_second_ad_start']


II. We will also drop the columns that are related to the columns that we have dropped or have no numerical or categorical siginificance. An example of this kind of data is the `naic_code` which is a code for the name of the job title. Since we have already included the actual title of the job, we decided to neglect the code. Similary, we have removed newspaper names of ads that the applicants used similar columns.

In [7]:
drop_list2 =  ['agent_city','agent_state','employer_address_1',\
        'employer_address_2','employer_country','employer_phone',\
        'employer_phone_ext','employer_postal_code','employer_city','foreign_worker_info_city',\
        'employer_decl_info_title','foreign_worker_info_inst','foreign_worker_info_state',\
        'foreign_worker_info_major','job_info_alt_combo_ed_exp','job_info_alt_field','job_info_combo_occupation',\
        'job_info_job_req_normal','job_info_major','job_info_training',\
        'job_info_job_title','job_info_work_city','job_info_work_postal_code','naics_2007_us_code',\
        'naics_code','naics_us_code', 'naics_us_code_2007',\
        'preparer_info_emp_completed','pw_determ_date', 'pw_expire_date',\
        'pw_level_9089', 'pw_soc_code','pw_soc_title', 'pw_source_name_9089',\
        'pw_track_num', 'rec_info_barg_rep_notified','recr_info_first_ad_start',\
        'recr_info_sunday_newspaper','recr_info_swa_job_order_end',\
        'recr_info_swa_job_order_start','ri_1st_ad_newspaper_name',\
        'ri_2nd_ad_newspaper_name','ri_2nd_ad_newspaper_or_journal',\
        'ri_employer_web_post_from', 'ri_employer_web_post_to',\
        'ji_live_in_dom_svc_contract','ri_local_ethnic_paper_from',\
        'ri_local_ethnic_paper_to', "job_info_work_state",\
        'ri_posted_notice_at_worksite','schd_a_sheepherder', 'us_economic_sector',\
        'wage_offer_from_9089','wage_offered_from_9089',\
        'wage_offer_to_9089','wage_offered_to_9089','employer_name',\
        'wage_offer_unit_of_pay_9089','wage_offered_unit_of_pay_9089','case_received_date','application_type'\
        ,'job_info_education','job_info_education_other']

In [8]:
# Drop the columns. 
new_data = data.drop(drop_list+drop_list2,axis=1)

## Merging Columns

This data set was pulled from various applications over the years, and the variables names were not consistent. Therefore, we will have to merge these variables into one column. First, we check to make sure the columns we want to merge are disjoint; if both columns have values in the same row, we will have to individually look at the row to see how to merge them correctly. In the codes below we verified that the columns are indeed disjoint. Then, we merged the columns together.

In [9]:
# Check to make sure all merging columns are disjoint
# This code will print the column name if they are not. 

for a,b in [["case_number","case_no"],\
            ["country_of_citizenship","country_of_citzenship"],\
            ["foreign_worker_info_education_other","fw_info_education_other"],\
            ["foreign_worker_yr_rel_edu_completed","fw_info_yr_rel_edu_completed"],\
            ["naics_2007_us_title","naics_title"],\
            ["naics_us_title","naics_us_title_2007"]]:
    
    c = new_data[a].isnull()*1
    d = new_data[b].isnull()*1
    if sum(c+d==0):
        print(a)

In [10]:
# Merge the data. 

for a,b in [["case_number","case_no"],\
            ["country_of_citizenship","country_of_citzenship"],\
            ["foreign_worker_info_education_other","fw_info_education_other"],\
            ["naics_title","naics_2007_us_title"],\
            ["naics_us_title","naics_us_title_2007"]]:
    
    new_data[a] = new_data[[a,b]].fillna('').sum(axis=1)
    new_data = new_data.drop([b],axis=1)
    

The NAICS Title had four different columns, so we had to merge two more columns together

In [11]:
# Merging the NAICS columns. 
new_data["naics_title"] = new_data[["naics_title","naics_us_title"]].fillna('').sum(axis=1)
new_data = new_data.drop(["naics_us_title"],axis=1)

`fw_info_yr_rel_edu_completed` and `foreign_worker_yr_rel_edu_completed` are disjoint as we can see in the cell above. However, since they contain floats instead of strings, they have to be merged a different way.

In [12]:
# Merging year of education columns.
new_data["foreign_worker_yr_rel_edu_completed"] = new_data[["fw_info_yr_rel_edu_completed", "foreign_worker_yr_rel_edu_completed"]].fillna(0)
new_data = new_data.drop(["fw_info_yr_rel_edu_completed"],axis=1)
new_data["foreign_worker_yr_rel_edu_completed"] = new_data["foreign_worker_yr_rel_edu_completed"].replace(0, np.nan)

We decided to drop any rows that contain data for only half of the variables.

In [13]:
# Identify how many rows contain only half the variables.
np.sum(new_data.isnull().sum(axis=1)>17)

174

In [14]:
# Drop the 239090 rows containing half the data.
new_data = new_data.dropna(axis=0, thresh=17)

## Standardize Column Information 

We have some columns that have mixed information. For example, `pw_unit_of_pay_9089` tells us whether the applicant's wage is yearly, hourly, etc. However, the inputs consist of `yr`, `mth`, `bi`, `hr`, `wk`, `Hour`, `Bi-Weekly`, `Month`, `Week`, `Year`. We standardized these to be `Hour`, `Bi-Weekly`, `Month`,  `Week`, `Year`. The employer state also have some states written in abreviation while others were spelled out. We changed all of the states to the abbreviations.

In [15]:
# Uniforming the pw_unit_of_pay_9089 rows. 
new_data["pw_unit_of_pay_9089"] = new_data["pw_unit_of_pay_9089"].replace("yr", "Year")
new_data["pw_unit_of_pay_9089"] = new_data["pw_unit_of_pay_9089"].replace("mth", "Month")
new_data["pw_unit_of_pay_9089"] = new_data["pw_unit_of_pay_9089"].replace("bi", "Bi-Weekly")
new_data["pw_unit_of_pay_9089"] = new_data["pw_unit_of_pay_9089"].replace("hr", "Hour")
new_data["pw_unit_of_pay_9089"] = new_data["pw_unit_of_pay_9089"].replace("wk", "Week")

In [16]:
# Creating state to abbreviation Dictionary. 
us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ',\
    'Arkansas': 'AR','California': 'CA', 'Colorado': 'CO','Connecticut': 'CT',\
    'Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI',\
    'Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA',\
    'Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME',\
    'Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',\
    'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE',\
    'Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',\
    'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH',\
    'Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI',\
    'South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',\
    'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA',\
    'West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',\
    'DISTRICT OF COLUMBIA': 'DC','VIRGIN ISLANDS' : 'VI','BRITISH COLUMBIA' :'BC',\
    'PUERTO RICO': 'PR','MARSHALL ISLANDS':'MH','NORTHERN MARIANA ISLANDS':'MP',\
    'GUAM':'GU'}
us_state_abbrev = {state.upper(): abrev for state, abrev in us_state_abbrev.items()}

# Convert to abbreviations.
new_data.loc[new_data["employer_state"].str.len()>2,"employer_state"]= new_data[new_data["employer_state"].str.len()>2]["employer_state"].replace(us_state_abbrev)

## Changing Variables Types

I. Turn time variables into pandas `datetime` objects. 

In [17]:
# Turning decision_date into datetime object. 
new_data['decision_date'] = pd.to_datetime(new_data['decision_date'],format = "%Y-%m-%d")

In [18]:
# Turn foreign_worker_yr_rel_edu_completed into datetime object.
mask1 = new_data['foreign_worker_yr_rel_edu_completed'] < 1900
mask2 = new_data['foreign_worker_yr_rel_edu_completed'].isnull()
new_data.loc[mask1,'foreign_worker_yr_rel_edu_completed'] = new_data.loc[mask1,'foreign_worker_yr_rel_edu_completed'] = 0
new_data.loc[mask1,'foreign_worker_yr_rel_edu_completed'] = new_data.loc[mask2,'foreign_worker_yr_rel_edu_completed'] = 0

new_data.foreign_worker_yr_rel_edu_completed = new_data.foreign_worker_yr_rel_edu_completed.astype(int)

mask3 = new_data['foreign_worker_yr_rel_edu_completed'] != 0
new_data.loc[mask3,'foreign_worker_yr_rel_edu_completed'] = pd.to_datetime(new_data.loc[mask3,'foreign_worker_yr_rel_edu_completed'],format='%Y')

a = pd.to_datetime(1970,format="%Y")
new_data['foreign_worker_yr_rel_edu_completed'] = new_data['foreign_worker_yr_rel_edu_completed'].replace(a,pd.NaT)

In [19]:
# Turn employer_yr_estab into datetime object. 
mask1 = new_data['employer_yr_estab'] < 1900
mask2 = new_data['employer_yr_estab'].isnull()
new_data.loc[mask1,'employer_yr_estab'] = new_data.loc[mask1,'employer_yr_estab'] = 0
new_data.loc[mask1,'employer_yr_estab'] = new_data.loc[mask2,'employer_yr_estab'] = 0

new_data.employer_yr_estab = new_data.employer_yr_estab.astype(int)

mask3 = new_data['employer_yr_estab'] != 0
new_data.loc[mask3,'employer_yr_estab'] = pd.to_datetime(new_data.loc[mask3,'employer_yr_estab'],format='%Y')

a = pd.to_datetime(1970,format="%Y")
new_data['employer_yr_estab'] = new_data['employer_yr_estab'].replace(a,pd.NaT)

II. Turn `pw_amount_9089` into floats. 

In [20]:
# Turning into float. 
new_data['pw_amount_9089'] = pd.to_numeric(new_data['pw_amount_9089'].str.replace(",",""))

III. Turn 'Y' or 'N' objects into boolean.

In [21]:
# Turning Yes or into booleans. 
for column in ['recr_info_coll_univ_teacher',
               'recr_info_employer_rec_payment',
               'recr_info_professional_occ',
               'refile',
               'ri_layoff_in_past_six_months',
               'ji_live_in_domestic_service',
               'job_info_foreign_ed',
               'job_info_foreign_lang_req',
               'job_info_experience']:
    
    new_data[column] = new_data[column].replace({'Y':True,'N':False})

IV. Turn None to NaN

In [22]:
new_data.foreign_worker_info_education = new_data['foreign_worker_info_education'].replace({"None":np.nan,"-":np.nan,"--------":np.nan})
new_data.foreign_worker_info_education_other = new_data['foreign_worker_info_education_other'].replace({"None":np.nan,"-":np.nan,"--------":np.nan,"-----------":np.nan,"--------------":np.nan})

V. Case Received date to year 

In [23]:
#Change decision_date to just year, then convert back to datetime object
new_data.decision_date = new_data.decision_date.dt.year
new_data['decision_date'] = pd.to_datetime(new_data['decision_date'],format = "%Y")

VI. Combine duplicate case_number data

In [24]:
data_frame = pd.concat([
        new_data[new_data.duplicated('case_number')],
        new_data.loc[new_data.drop_duplicates('case_number',keep=False).index]
    ])
data_frame = pd.concat([
        data_frame[data_frame.duplicated('case_number')],
        data_frame.loc[data_frame.drop_duplicates('case_number',keep=False).index]
    ])

## Cleaned Data

Lastly, we set the index of our dataframe to be the case number of each applicant. 

In [25]:
data_frame = data_frame.set_index(data_frame.case_number)
data_frame = data_frame.drop('case_number',axis = 1)

Our data is finally cleaned. It has 33 variables and 239090 data points.

# Potential Problems

## Evaluate Sources
The dataset comes from the US Department of Labor. They disclose the US permanent visa application information every year. Since this dataset comes from the US Department of Labor, we believe that it is a reliable dataset and that the information given are accurate. We also think that there should not be a bias other than people filling out their information wrong due to misunderstanding of the question. For the rows where the value does not make sense for a given variable, we have removed the whole datapoint or their response. 

Although we have removed 135272 datapoints from our dataset, we still have datapoints with missing information. We have already done a little bit of initial feature engineering. For example, we created a new column indicating whether the datapoint has information on using an agent or not. For the other datapoints, we do not think this will affect our results. 

We have already changed all datapoints with wrong information in our data cleaning process.

## Suitability
In our proposal, we wanted answer the following questions:

1) What is the likelihood of an individual getting certified for a US Permanent Visa given their background?

2) Which columns/variables have significant negative effects on receiving a US Permanent Visa?

3) Which columns/variables have significant positive effects on receiving a US Permanent Visa?

4) Is there bias in receiving a permanent visa? 

We believe that our dataset can answer these questions very well. We have lots of variables to work with, and our response variable is very clear. Looking at the likelihood of an individual receiving a visa can be answered by looking at the trends of our variables and their application results. As for positive and negative effects of variables, we can see which variables most commonly lead to a denied status versus a certified status. To determine bias, we will look within categories such as job title or job region and determine whether people of certain ages or origin have higher chances of being accepted or denied a permanent visa. We can also build statistical models to help us answer these questions.

# FEATURE ENGINEERING

## Remove Withdrawn Applications
Currently our response variable has the values Certified, Withdrawn, Certified-Expired, and Denied. The rows where the application has been withdrawn are not helpful for our purpose, since we are trying understand the likelihood of a person being certified. Therefore, we have removed all of the rows where the application has been withdrawn. The Certified-Expired label means that the person was certified for the permanent labor certifications but did not file a 140 within a 6 month period, meaning that their permanent visa expired. Since the individuals whose applicatios was Certified-Expired were certified at one point, we decided to group them with the other Certified applications. 

In [26]:
# Drop rows that have withdrawn -- don't need them for our analysis
data_frame = data_frame[data_frame["case_status"] != 'Withdrawn']

In [27]:
#Replaced "Certified-Expired" with "Certified"
data_frame["case_status"] = data_frame["case_status"].replace('Certified-Expired', 'Certified')
data_frame = data_frame.replace("", np.nan)

## Adding Medical School to Education 
In the column `foreign_worker_info_education`, applicants indicated the level of their education. If their highest level of education was not High Schoool, Bachelor's, Master's, Doctorate, or Associate's, they may indicate Other in that column. In that case, there is another column `foreign_worker_info_education_other` where those individuals can specify what type of education was completed. Since almost every individual who filled in this column has a different answer, this row may not be very helpful for our analysis. Looking at the `foreign_worker_info_education_other` column, we noticed that there were a lot of people with medical degrees. We thought that it would be interesting to analyze that degree type also. Hence, we used regular expressions to find the rows that indicated that they got a medical degree and added it to the `foreign_worker_info_education` column. We then deleted the `foreign_worker_info_education_other` column. 

In [28]:
# Find Medical Degree individuals. 
expr = re.compile(r".*M.D.*|.*MED.*|.*MD.*")
data_frame["foreign_worker_info_education_other"] = \
    data_frame["foreign_worker_info_education_other"].replace(expr, "Medical Degree")

In [29]:
# Add Medical Degree in education column.
mask = data_frame["foreign_worker_info_education_other"] == "Medical Degree"
data_frame.loc[mask,"foreign_worker_info_education"] = \
    data_frame[mask]["foreign_worker_info_education"].replace("Other", "Medical Degree")
data_frame = data_frame.drop("foreign_worker_info_education_other", axis=1)

##  If Applicants Used Agency or Not
This data set also contains information about the specific agencies of certain applicants. Agency information was given for those applicants who used an agency during the application process. This information is not very important to our study. We change this value to a boolean variable, with a `True` signifying an agency was used, or a `False` signifying no agency was used. We then dropped the `agent_firm_name` column.

In [30]:
data_frame["used_agency"] = ~data_frame["agent_firm_name"].isnull()
data_frame = data_frame.drop(["agent_firm_name"], axis=1)

In [31]:
data_frame[200100:200104]

,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_state,employer_yr_estab,foreign_worker_info_education,foreign_worker_yr_rel_edu_completed,ji_live_in_domestic_service,...,job_info_foreign_lang_req,naics_title,pw_amount_9089,pw_unit_of_pay_9089,recr_info_coll_univ_teacher,recr_info_employer_rec_payment,recr_info_professional_occ,refile,ri_layoff_in_past_six_months,used_agency
case_number,,,,,,,,,,,,,,,,,,,,,
A-16161-20020,Certified,NaN,TAIWAN,2016-01-01,59.0,NJ,2003-01-01,Bachelor's,1996-01-01,False,...,False,Custom Computer Programming Services,60840.0,Year,False,False,True,NaN,False,True
A-16161-20091,Certified,H-1B,SOUTH KOREA,2016-01-01,16.0,CO,2009-01-01,Bachelor's,1997-01-01,False,...,False,Medical Equipment and Supplies Manufacturing,99382.0,Year,False,False,True,NaN,False,True
A-16035-69181,Denied,H-1B,INDIA,2016-01-01,15000.0,IL,NaT,Master's,1996-01-01,False,...,False,Pharmaceutical Preparation Manufacturing,158579.0,Year,False,False,True,NaN,False,True
A-16181-27351,Certified,H-1B,INDIA,2016-01-01,200.0,TX,1997-01-01,Bachelor's,2006-01-01,False,...,False,Custom Computer Programming Services,170456.0,Year,False,False,True,NaN,False,False


In [32]:
data_frame.to_csv("cleaned_data.csv")

In [33]:
data_frame.shape

(228644, 23)